In [1]:
import random
random.seed(218)

In [2]:
alphabet = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])

def generate_combinations(word):
    # num_combinations_per_word = num_combinations(len(word)) + 1
    word = word.strip()
    num_combinations = 7

    combinations = set()

    characters = set()
    for c in word:
        characters.add(c)

    # while len(combinations) < num_combinations_per_word:
    #     indexes = random.sample(range(len(word)), random.randint(1, max_blanks))
    #     new_word = list(word)

    #     for idx in indexes:
    #         new_word[idx] = '_'
    #     combinations.add(''.join(new_word))

    for i in range(1, len(characters) + 1):
        for j in range(num_combinations):

            removal_characters = random.sample(list(characters), k = i)
            new_word = word

            for c in removal_characters:
                new_word = new_word.replace(c, '_')
            combinations.add(new_word)

    return list(combinations)

def create_hangman_data(data_file_path, input_file_path, target_file_path, wrong_guesses_file_path):
    with open(data_file_path, 'r') as f:
        words = f.read().splitlines()

    input_data = []
    target_data = []
    wrong_guesses = []
    maxlen = 0

    for word in words:
        combinations = generate_combinations(word)

        total_characters = set(list(word.strip()))
        total_wrong_guesses = alphabet.difference(total_characters)

        for combination in combinations:
            for i in range(0, 12):
                cur_wrong_guesses = random.sample(list(total_wrong_guesses), k = min(i, 26 - len(set(list(word.strip())))))
                cur_wrong_guesses = ''.join(cur_wrong_guesses)
                input_data.append(combination)
                target_data.append(word)
                wrong_guesses.append(cur_wrong_guesses)
        
        maxlen = max(maxlen, len(word))
    
    shuffled_idx = random.sample(range(len(input_data)), k = len(input_data))
    input_data = [input_data[i] for i in shuffled_idx]
    target_data = [target_data[i] for i in shuffled_idx]
    wrong_guesses = [wrong_guesses[i] for i in shuffled_idx]

    with open(input_file_path, 'w') as f:
        f.write('\n'.join(input_data))

    with open(target_file_path, 'w') as f:
        f.write('\n'.join(target_data))
    
    with open(wrong_guesses_file_path, 'w') as f:
        f.write('\n'.join(wrong_guesses))

    return len(input_data), maxlen

In [3]:
data_file_path = "words_250000_train.txt"
input_file_path = "input_train.txt"
target_file_path = "target_train.txt" 
wrong_guesses_file_path = "wrong_guesses.txt"

In [77]:
data_size, MAX_SEQ_LEN = create_hangman_data(data_file_path=data_file_path, input_file_path=input_file_path, target_file_path=target_file_path, wrong_guesses_file_path=wrong_guesses_file_path)

In [ ]:
print("Size of input training data is %d" % data_size)
print("Max length of a word is %d" % MAX_SEQ_LEN)

In [22]:
char2idx = {'<PAD>' : 0}
idx2char = ['<PAD>']

for i in range(26):
    char2idx[chr(i + ord('a'))] = i + 1
    idx2char.append(chr(ord('a') + i))

char2idx['_'] = 27
idx2char.append('_')

In [23]:
def get_coded_sequence(word):
    word = word.strip()
    seq = list(word)
    seq = seq + ['<PAD>'] * (MAX_SEQ_LEN - len(seq))
    for idx, c in enumerate(seq):
        seq[idx] = char2idx[c]
    
    return seq

In [25]:
alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
def data_generator(input_file, target_file, wrong_guesses_file):
    with open(input_file) as f_input, open(target_file) as f_target, open(wrong_guesses_file) as f_wguess:
        for input_word, target_word, wrong_guess in zip(f_input, f_target, f_wguess):
            input_tokens = get_coded_sequence(input_word)
            target_tokens = np.zeros(26)
            incorrect_tokens = np.zeros(26)
            correct_tokens = np.zeros(26)

            present_chars = set(list(input_word.strip()))
            missing_chars = set(list(target_word.strip()))
            missing_chars = missing_chars.difference(present_chars)
            for c in missing_chars:
                target_tokens[ord(c) - ord('a')] = 1
            
            for c in present_chars:
                if c == '_':
                    continue
                correct_tokens[ord(c) - ord('a')] = 1

            for c in list(wrong_guess.strip()):
                incorrect_tokens[ord(c) - ord('a')] = 1

            yield (input_tokens, correct_tokens, incorrect_tokens), target_tokens
            

            

In [26]:
import tensorflow as tf
import numpy as np
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.clear_session()

In [27]:
BATCH_SIZE = 2048
MAX_SEQ_LEN = 29

In [28]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(input_file_path, target_file_path, wrong_guesses_file_path),
    output_signature = (
        (tf.TensorSpec(shape = (MAX_SEQ_LEN,), dtype = tf.int32), tf.TensorSpec(shape = (26,), dtype = tf.int32), tf.TensorSpec(shape = (26,), dtype = tf.int32)),
        tf.TensorSpec(shape = (26,), dtype = tf.int32)
    )
)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder = False, num_parallel_calls = tf.data.AUTOTUNE)

In [13]:
from keras import Input, Model
from keras.layers import LSTM, Bidirectional, Embedding, Dense, TimeDistributed, Attention, Concatenate, Masking, GlobalMaxPooling1D, Dropout, GlobalAveragePooling1D
from keras.optimizers import Adam

In [14]:
adam_opt = Adam(learning_rate = 0.001)

In [18]:
LATENT_DIM = 256

input1 = Input(shape=(MAX_SEQ_LEN,), name = "Input_Word")

embedding_layer = Embedding(input_dim = 28, output_dim = 40, mask_zero = True, name = "Embedding_Layer")
emb_input = embedding_layer(input1)

bilstm_layer = Bidirectional(LSTM(LATENT_DIM, return_sequences = True, name = "Bidirection_LSTM_Layer", dropout = 0.1))
lstm_output = bilstm_layer(emb_input)

pool_layer = GlobalMaxPooling1D(name = "MaxPooling_Layer")
lstm_output = pool_layer(lstm_output)

input2 = Input(shape = (26,), name = "Correct_Guesses")
input3 = Input(shape = (26,), name = "Incorrect_Guesses")


dense_layer = Dense(128, activation = 'relu', name = "Dense_Layer")
dense_layer1 = Dense(128, activation = 'relu', name = "Dense_Layer1")
correct_guess_output = dense_layer(input2)
correct_guess_output = Dropout(0.1, seed = 218)(correct_guess_output)
incorrect_guess_output = dense_layer1(input3)
incorrect_guess_output = Dropout(0.1, seed = 218)(incorrect_guess_output)


concat_layer = Concatenate(name = "Concatenate_Layer")
guess_output = concat_layer([correct_guess_output, incorrect_guess_output])

dense_layer2 = Dense(256, activation = 'relu', name = "Dense_Layer2")
guess_output = dense_layer2(guess_output)
guess_output = Dropout(0.2, seed = 218)(guess_output)


concat_output = concat_layer([lstm_output, guess_output])

dense_layera = Dense(512, activation='relu', name = "Dense_LayerA")
dense_layerb = Dense(256, activation='relu', name = "Dense_LayerB")
dense_layerc = Dense(26, activation='sigmoid', name = "Dense_LayerC")
output = dense_layera(concat_output)
output = Dropout(0.3, seed = 218)(output)
output = dense_layerb(output)
output = Dropout(0.2, seed = 218)(output)
output = dense_layerc(output)

model = Model([input1, input2, input3], output)

model.compile(optimizer = adam_opt, loss='binary_focal_crossentropy', metrics=['acc'])

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Correct_Guesses (InputLayer)   [(None, 26)]         0           []                               
                                                                                                  
 Incorrect_Guesses (InputLayer)  [(None, 26)]        0           []                               
                                                                                                  
 Dense_Layer (Dense)            (None, 128)          3456        ['Correct_Guesses[0][0]']        
                                                                                                  
 Dense_Layer1 (Dense)           (None, 128)          3456        ['Incorrect_Guesses[0][0]']      
                                                                                            

In [107]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model_checkpoint = ModelCheckpoint("checkpoints/weights.{epoch:02d}-{acc:.4f}.hdf5", monitor = "acc")
early_stopping = EarlyStopping(monitor='acc', patience=2, restore_best_weights = True)
callbacks_list = [model_checkpoint, early_stopping]

In [ ]:
model.fit(train_dataset, epochs = 5, callbacks = callbacks_list, verbose = 1)

In [ ]:
model.save('model_weights.h5')